# Multigraphes orientés et pondérés par le temps

In [63]:
import networkx as nx
import matplotlib.pyplot as plt

## Fonctions usuelles sur les graphes

In [89]:
def createGraphFromFile(filename):
    "reads textfile and creates the corresponding directed multigraph"

    G = nx.MultiDiGraph()
    
    with open(filename, encoding='utf8') as f:
        for line in f: 
            if line[0] == 's':
                G.add_node(int(line[7]))
            if line[0] == 'a':
                u,v,t,l = line[7], line[9], line[11], line[13]
                G.add_edge(int(u), int(v), date=int(t), duration=int(l))
    return G

In [90]:
MG = createGraphFromFile("graphe.txt")

In [91]:
# Display agencies 
for k in MG.edges(data=True) :
    print(k)

(1, 2, {'date': 1, 'duration': 1})
(1, 2, {'date': 2, 'duration': 1})
(2, 3, {'date': 5, 'duration': 1})


In [92]:
def createGraphFromMultiGraph(MG):
    G = nx.DiGraph()
    for s in MG.nodes():
        for u,v,attribute in MG.edges(data=True):
            t = attribute['date']
            l = attribute['duration']
            if s == u:
                uOut = (u, t)
                vIn = (v, t+l)
                if uOut not in G.nodes():
                    G.add_node(uOut)
                if vIn not in G.nodes():
                    G.add_node(vIn)
                G.add_edge(uOut, vIn, weight=1)
                for i,j in G.nodes():
                    if i == u:
                        if j > t:
                            G.add_edge(uOut, (i,j), weight=0)
                        elif j < t:
                            G.add_edge((i,j), uOut, weight=0)
    
    return G

In [93]:
G = createGraphFromMultiGraph(MG)

In [95]:
print(G.edges(data=True))

[((1, 1), (2, 2), {'weight': 1}), ((1, 1), (1, 2), {'weight': 0}), ((2, 2), (2, 5), {'weight': 0}), ((1, 2), (2, 3), {'weight': 1}), ((2, 3), (2, 5), {'weight': 0}), ((2, 5), (3, 6), {'weight': 1})]


In [119]:
def createMultiGraph(vertexNumber, duration):
    """generate a directed multigraph given a number of vertices 
    and a duration with a fixed pattern"""
    
    G = nx.MultiDiGraph()
        
    # Vertices
    for i in range(1, vertexNumber+1):
        G.add_node(i)
    
    # Edges
    date = 1
    v = 1
    while v != vertexNumber:
        G.add_edge(v, v+1, date=date, duration=duration)
        if v+2 <= vertexNumber:
            G.add_edge(v, v+2, date=date+1, duration=duration)
        date += 1
        v += 1
    
    return G

In [120]:
MG = createMultiGraph(5,1)

In [121]:
for v in MG.nodes:
    print(v)
for e in MG.edges(data=True):
    print(e)

1
2
3
4
5
(1, 2, {'date': 1, 'duration': 1})
(1, 3, {'date': 2, 'duration': 1})
(2, 3, {'date': 2, 'duration': 1})
(2, 4, {'date': 3, 'duration': 1})
(3, 4, {'date': 3, 'duration': 1})
(3, 5, {'date': 4, 'duration': 1})
(4, 5, {'date': 4, 'duration': 1})


## Algorithmes

#### IV Plus court chemin

*Note : On utilise Bellman-Ford, sans se préoccuper des cycles négatifs*

In [134]:
def shortestPath(G, departure, arrival):
    """returns the shortest path, meaning the path containing 
    the least amount of edges between 'departure' and 'arrival'."""
    
    # error detection
    if departure not in G or arrival not in G:
        err = f"Either departure {departure} or arrival {arrival} is not in G"
        raise nx.NodeNotFound(err)
    
    if departure == arrival:
        return (0, [departure])
    
    dist = dict()
    for i in G.nodes():
        dist[i] = (float("Inf"), -1)
    dist[departure] = (0,-1)
    
    # Get distances 
    for u,v,w in G.edges(data='weight'):
        d1, p1 = dist[u]
        d2, p2 = dist[v]
        if d1 != float("Inf") and d1+w < d2:
            dist[v] = (d1+w, u)
    
    d,p = dist[arrival]
    if d == float("Inf"):
        err = f"There is no path between departure {departure} and arrival {arrival}"
        raise nx.PathNotFound(err)
    
    distance = d
    # Get path
    v = arrival
    path = [v]
    while v != departure:
        d, p = dist[v] 
        path = [p] + path
        v = p

    return distance, path

In [135]:
# Test
print(shortestPath(G, (1,1), (3,6)))

(2, [(1, 1), (2, 2), (2, 5), (3, 6)])


## Tests